<a href="https://colab.research.google.com/github/j1h318/KSCO_Graph-Embedding/blob/main/Node2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install neo4j
!pip install neo4j-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.14.1-py3-none-any.whl size=267560 sha256=b0fc04714e3ed680221fb5eb640de0862606e7925a829dd7604a57f89c3bc9aa
  Stored in directory: /root/.cache/pip/wheels/13/01/ff/de2142e172dafcd475f118499db2140c66562e42fd500c460a
Successfully built neo4j
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.14.1-py3-none-any.whl size=267778 sha256=fc56ebef3b8c414d983151ac1ff1e9a23b5bdc25c9b762ff2170e1f01217f982
  Stored in directory: /root/.cach

In [3]:
from neo4j import GraphDatabase
from sklearn.manifold import TSNE
import numpy as np
import altair as alt
import pandas as pd

In [4]:
URI = "neo4j+s://[URL].databases.neo4j.io"
driver = GraphDatabase.driver(URI, auth=("neo4j", "[PWD]"))

In [9]:
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = "neo4j+s://[URL].databases.neo4j.io"
        self.__user = "neo4j"
        self.__pwd = "[PWD]"
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [10]:
uri = "neo4j+s://[URL].databases.neo4j.io"
pwd = "[PWD]"
conn = Neo4jConnection(uri=uri, user="neo4j", pwd=pwd)
conn.query('MATCH (n) RETURN COUNT(n) AS count')

[<Record count=94970>]

# Node2vec를 위한 임베딩 결과


In [12]:
query = '''MATCH (n:OCCUPATION) RETURN n.occu1Cd As jobco, n.occu2Cd AS jobs, n.embedding AS embedding;
'''
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df.head()

,jobco,jobs,embedding
0,01,011,"[0.23874381184577942, 0.024699093773961067]"
1,01,012,"[0.23851986229419708, -0.05143991485238075]"
2,01,013,"[0.23847506940364838, 0.033332280814647675]"
3,01,014,"[0.2386094480752945, -0.2209843099117279]"
4,01,015,"[0.2385646551847458, -0.13621211051940918]"


In [13]:
df

,jobco,jobs,embedding
0,01,011,"[0.23874381184577942, 0.024699093773961067]"
1,01,012,"[0.23851986229419708, -0.05143991485238075]"
2,01,013,"[0.23847506940364838, 0.033332280814647675]"
3,01,014,"[0.2386094480752945, -0.2209843099117279]"
4,01,015,"[0.2385646551847458, -0.13621211051940918]"
...,...,...,...
504,None,None,"[0.21473601460456848, -0.03740270808339119]"
505,None,None,"[0.21451206505298615, -0.11354170739650726]"
506,None,None,"[0.21446727216243744, -0.028769519180059433]"
507,None,None,"[0.21460165083408356, 0.21691389381885529]"


In [14]:
tsne = TSNE(n_components=2)
embedding_array = np.array([np.array(embedding) for embedding in df.embedding])
embedding_array

array([[ 0.23874381,  0.02469909],
       [ 0.23851986, -0.05143991],
       [ 0.23847507,  0.03333228],
       ...,
       [ 0.21446727, -0.02876952],
       [ 0.21460165,  0.21691389],
       [ 0.21455686, -0.19831391]])

In [15]:
X_embeded = tsne.fit_transform(embedding_array)

In [16]:
jobs = df.jobs
tsne_df = pd.DataFrame(data = {
    "job": jobs,
    "jobs" : df.jobco,
    "x": [value[0] for value in X_embeded],
    "y": [value[1] for value in X_embeded]
})
tsne_df.head()

,job,jobs,x,y
0,011,01,-3.653775,-8.002926
1,012,01,-9.648104,1.385987
2,013,01,-2.928489,-8.898936
3,014,01,-23.702669,19.570536
4,015,01,-15.275319,11.864348


In [18]:
alt.Chart(tsne_df).mark_circle(size=60).encode(
    x='x',
    y='y',
).properties(width=700, height=400)

alt.Chart(...)